# Hydrostatic Equilibrium Solver

In [20]:
using Pkg
Pkg.activate("..")

using GLMakie
using NonRelativisticMergers
using LinearAlgebra
using LinearSolve
using NonlinearSolve

import NonRelativisticMergers: solve_poisson

  Activating project at `~/gitrep/NonRelativisticMergers`


**Goal:** Solve the hydrostatic equation
$$
\nabla P = - \rho \nabla \phi \; ,
$$
numerically. This will be needed to obtain the initial condition for the merger simulation.

We start from some density distribution $\rho(x)$ and find the gravitational potential $\phi(x)$ by solving the Poisson equation
$$
\nabla^2 \phi = 4\pi G \rho \; .
$$

### 1d case

In 1d we have to solve
$$
\partial_x P(x) = - \rho(x) \partial_x \phi(x) \; .
$$

We could solve this by direct integration, but this will not be possible for the 2d case. Instead we can use finite difference and a non-linear solver to find the expression of $P(x)$. 

In [33]:
(; xl, Nx) = gd = Grid1D(; xmin=-1, xmax=1, Nx=500)
prob = FDProblem(gd, EulerSelfGravity(), KT(), HLLC())

ρ = [0.1 + 1.8 * exp(-100 * x^2) for x in xl]
ϕ = solve_poisson(prob, ρ);

In [34]:
∇x = Matrix(Tridiagonal(ones(Nx - 1), zeros(Nx), -ones(Nx - 1)))
∇x[1, Nx] = 1
∇x[Nx, 1] = -1

∇ϕ = ∇x * ϕ;

In [37]:
nlprob = NonlinearProblem((P, _) -> ∇x * P .+ ρ .* ∇ϕ, zeros(Nx))
nlsol = NonlinearSolve.solve(nlprob)
P = nlsol.u

500-element Vector{Float64}:
 -0.006379837479297162
 -0.006379833798117487
 -0.006379826429842946
 -0.006379815362636525
 -0.006379800578716267
 -0.006379782054330686
 -0.006379759759717205
 -0.006379733659057125
 -0.006379703710415471
 -0.006379669865676651
  ⋮
 -0.00637970371041575
 -0.006379733659056898
 -0.006379759759717407
 -0.006379782054330513
 -0.006379800578716566
 -0.006379815362636275
 -0.006379826429843247
 -0.006379833798117204
 -0.00637983747929745

In [35]:
lalg = SVDFactorization()
# lalg = QRFactorization()

lprob = LinearProblem(∇x, -ρ .* ∇ϕ)
lsol = LinearSolve.solve(lprob, lalg)
P = lsol.u

500-element Vector{Float64}:
 -0.006372686045867099
 -0.007320575341375153
 -0.0063726749964128845
 -0.007320556905894192
 -0.006372649145286201
 -0.007320523597588339
 -0.006372608326287138
 -0.00732047520231479
 -0.00637255227698541
 -0.007320411408934319
  ⋮
 -0.0073204452536734105
 -0.006372582225626835
 -0.0073205013029750705
 -0.006372630620900453
 -0.007320542121974229
 -0.006372663929206213
 -0.007320567973100909
 -0.006372682364687143
 -0.007320579022555118

In [38]:
f = lines(ρ, label="rho")
lines!(ϕ, label="phi")
lines!(P, label="P")
axislegend()
f

### 2d case

In 2d we have to solve the following equations for $P(x,y)$
$$
\begin{cases}
    \partial_x P(x,y) + \rho(x,y) \partial_x \phi(x,y) = 0 \; , \\
    \partial_y P(x,y) + \rho(x,y) \partial_y \phi(x,y) = 0 \; .
\end{cases}
$$

In [27]:

function solve_hydrostatic(gd::Grid1D, ρ, ϕ)
    (; Nx) = gd

    P = zeros(size(ρ))

    ∇x = Matrix(Tridiagonal(ones(Nx - 1), zeros(Nx), -ones(Nx - 1)))
    ∇x[1, Nx] = 1
    ∇x[Nx, 1] = -1

    return P
end

solve_hydrostatic (generic function with 1 method)

In [5]:
(; xl, yl, Nx) = gd = Grid2D(; xmin=-2, xmax=2, ymin=-2, ymax=2, Nx=100, Ny=100)
prob = FDProblem(gd, EulerSelfGravity(), KT(), HLLC())

ρ = [0.0 + exp(-10 * (x^2 + y^2)) for x in xl, y in yl]
ϕ = solve_poisson(prob, ρ);

In [3]:
f = Figure()
Axis(f[1, 1])
heatmap!(xl, yl, ρ)
Axis(f[1, 2])
heatmap!(xl, yl, ϕ)
# Axis(f[1, 3])
# heatmap!(xl, yl, P)
f